In [ ]:
!pip install flask flask-sqlalchemy flask-session flask-login
!pip install qwen-vl-utils transformers accelerate pdf2image llm-axe
!pip install faiss-cpu sentence-transformers numpy
!pip install --ignore-installed flask
!apt update
!apt install poppler-utils -y

In [ ]:
# from flask import Flask, request, redirect, url_for, render_template, flash, session
# from flask_login import LoginManager, UserMixin, login_user, login_required, logout_user, current_user
# from flask_sqlalchemy import SQLAlchemy
# from flask_session import Session
# import os

# app = Flask(__name__)
# app.secret_key = 'dishuKB@852'

# # Configure database
# app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///users.db'
# app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

# # Configure server-side sessions
# app.config['SESSION_TYPE'] = 'filesystem'
# Session(app)

# # Initialize database and Flask-Login
# db = SQLAlchemy(app)
# login_manager = LoginManager()
# login_manager.init_app(app)
# login_manager.login_view = 'login'

# # User model for database
# class User(db.Model, UserMixin):
#     id = db.Column(db.Integer, primary_key=True)
#     username = db.Column(db.String(150), unique=True, nullable=False)
#     password = db.Column(db.String(150), nullable=False)
#     session_token = db.Column(db.String(150), nullable=True)

# # Create the database tables if they don't exist
# with app.app_context():
#     db.create_all()

# # Flask-Login user loader
# @login_manager.user_loader
# def load_user(user_id):
#     return User.query.get(int(user_id))

# # Routes
# @app.route('/login', methods=['GET', 'POST'])
# def login():
#     if request.method == 'POST':
#         username = request.form['username']
#         password = request.form['password']
#         user = User.query.filter_by(username=username).first()

#         if user and user.password == password:
#             login_user(user)
#             session['session_token'] = os.urandom(24).hex()  # Generate a session token
#             user.session_token = session['session_token']
#             db.session.commit()
#             flash('Logged in successfully.')
#             return redirect(url_for('protected'))
#         else:
#             flash('Invalid username or password.')
#     return render_template('login.html')

# @app.route('/protected')
# @login_required
# def protected():
#     return f'Hello, {current_user.username}! You are logged in.'

# @app.route('/logout')
# @login_required
# def logout():
#     current_user.session_token = None
#     db.session.commit()
#     logout_user()
#     session.pop('session_token', None)
#     flash('Logged out successfully.')
#     return redirect(url_for('login'))

# @app.before_request
# def check_session():
#     # Check if the user is authenticated via session token
#     if 'session_token' in session:
#         user = User.query.filter_by(session_token=session['session_token']).first()
#         if user:
#             login_user(user)

# @app.route('/register', methods=['GET', 'POST'])
# def register():
#     if request.method == 'POST':
#         username = request.form['username']
#         password = request.form['password']
#         if User.query.filter_by(username=username).first():
#             flash('Username already exists.')
#         else:
#             new_user = User(username=username, password=password)
#             db.session.add(new_user)
#             db.session.commit()
#             flash('User registered successfully.')
#             return redirect(url_for('login'))
#     return render_template('register.html')

# if __name__ == '__main__':
#     app.run(host='0.0.0.0', port=5000)

In [2]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
from accelerate import Accelerator
import json
import sqlite3
from datetime import datetime
from llm_axe.agents import OnlineAgent
from pathlib import Path
import os
from flask import Flask, request, jsonify, send_file, abort
from werkzeug.utils import secure_filename
import torch

In [3]:
accelerator = Accelerator()
device = accelerator.device
print(device)

cuda


In [4]:
# default: Load the model on the available device(s)
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-7B-Instruct", torch_dtype="auto", device_map="auto"
)

# We recommend enabling flash_attention_2 for better acceleration and memory saving, especially in multi-image and video scenarios.
# model = Qwen2VLForConditionalGeneration.from_pretrained(
#     "Qwen/Qwen2-VL-7B-Instruct",
#     torch_dtype=torch.bfloat16,
#     attn_implementation="flash_attention_2",
#     device_map="auto",
# )

# default processer
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-7B-Instruct")

# The default range for the number of visual tokens per image in the model is 4-16384. You can set min_pixels and max_pixels according to your needs, such as a token count range of 256-1280, to balance speed and memory usage.
# min_pixels = 256*28*28
# max_pixels = 1280*28*28
# processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-7B-Instruct", min_pixels=min_pixels, max_pixels=max_pixels)

`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [5]:
class SQLiteFileSummaryStorage:
    """Store file summaries using SQLite"""
    
    def __init__(self, db_path="file_summaries.db"):
        self.db_path = db_path
        self._ensure_database_exists()
    
    def _ensure_database_exists(self):
        """Create database and table if they don't exist"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS file_summaries (
                filename TEXT PRIMARY KEY,
                filepath TEXT,
                summary TEXT,
                timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
            )
        ''')
        
        conn.commit()
        conn.close()
    
    def save_summary(self, filename, filepath, summary):
        """Save a file summary"""
        try:
            conn = sqlite3.connect(self.db_path)
            cursor = conn.cursor()
            
            cursor.execute('''
                INSERT OR REPLACE INTO file_summaries (filename, filepath, summary)
                VALUES (?, ?, ?)
            ''', (filename, filepath, summary))
            
            conn.commit()
            conn.close()
            return True
        except Exception as e:
            print(f"Error saving summary: {e}")
            return False
    
    def get_summary(self, filename):
        """Retrieve a file summary"""
        try:
            conn = sqlite3.connect(self.db_path)
            cursor = conn.cursor()
            
            cursor.execute('''
                SELECT summary FROM file_summaries
                WHERE filename = ?
            ''', (filename,))
            
            result = cursor.fetchone()
            conn.close()
            
            return result[0] if result else None
        except Exception as e:
            print(f"Error retrieving summary: {e}")
            return None
    
    def get_all_summaries(self):
        """Get all stored summaries"""
        try:
            conn = sqlite3.connect(self.db_path)
            cursor = conn.cursor()
            
            cursor.execute('''
                SELECT filename, filepath, summary, timestamp 
                FROM file_summaries
            ''')
            
            results = cursor.fetchall()
            conn.close()
            
            return {row[0]: {
                'filepath': row[1],
                'summary': row[2],
                'timestamp': row[3]
            } for row in results}
        except Exception as e:
            print(f"Error retrieving summaries: {e}")
            return {}


# # Example usage:
# def main():
#     # Example using JSON storage
#     json_storage = JSONFileSummaryStorage()
    
#     # Save some summaries
#     json_storage.save_summary("document1.pdf", "This is a report about quarterly earnings")
#     json_storage.save_summary("image1.jpg", "A landscape photo of mountains")
    
#     # Retrieve a specific summary
#     print(json_storage.get_summary("document1.pdf"))
    
#     # Get all summaries
#     print(json_storage.get_all_summaries())
    
#     # Example using SQLite storage
#     sqlite_storage = SQLiteFileSummaryStorage()
    
#     # Save some summaries
#     sqlite_storage.save_summary("document1.pdf", "This is a report about quarterly earnings")
#     sqlite_storage.save_summary("image1.jpg", "A landscape photo of mountains")
    
#     # Retrieve a specific summary
#     print(sqlite_storage.get_summary("document1.pdf"))
    
#     # Get all summaries
#     print(sqlite_storage.get_all_summaries())

In [6]:
def pdf_to_images(pdf_path, output_folder):
    
    for i, page in enumerate(images):
        page.save(f"{output_folder}/page_{i + 1}.png", "PNG")

# # Example usage
# pdf_to_images("example.pdf", "output_images_folder")


In [7]:
def is_correct_format(messages: list[dict]) -> bool:
    """
    Checks if the messages are already in the correct format.
    
    Correct format:
    [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "message text"
                }
            ]
        }
    ]
    
    Args:
        messages (list[dict]): List of message dictionaries to validate
        
    Returns:
        bool: True if format is correct, False otherwise
    """
    try:
        if not isinstance(messages, list):
            return False
            
        for message in messages:
            # Check if basic structure exists
            if not isinstance(message, dict):
                return False
                
            if 'role' not in message or 'content' not in message:
                return False
                
            # Check role
            if not isinstance(message['role'], str):
                return False
                
            # Check content structure
            content = message['content']
            if not isinstance(content, list):
                return False
                
            # Validate each content item
            for item in content:
                if not isinstance(item, dict):
                    return False
                    
                if 'type' not in item or 'text' not in item:
                    return False
                    
                if item['type'] != 'text':
                    return False
                    
                if not isinstance(item['text'], str):
                    return False
                    
        return True
        
    except Exception:
        return False

def convert_message_format(messages: list[dict]) -> list[dict]:
    """
    Converts messages from simple format to the structured format required by the model.
    
    Input format:
    [
        {
            "role": "user",
            "content": "message text"
        }
    ]
    
    Output format:
    [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "message text"
                }
            ]
        }
    ]
    
    Args:
        messages (list[dict]): List of message dictionaries in simple format
        
    Returns:
        list[dict]: List of message dictionaries in structured format
    """
    converted_messages = []
    
    for message in messages:
        # Handle case where content is already in the correct format
        if isinstance(message.get('content'), list):
            converted_messages.append(message)
            continue
            
        # Convert simple string content to structured format
        converted_message = {
            "role": message["role"],
            "content": [
                {
                    "type": "text",
                    "text": message["content"]
                }
            ]
        }
        converted_messages.append(converted_message)
    
    return converted_messages

In [8]:
class JARVIS():

    # Your ask function will always receive a list of prompts
    # The prompts are in open ai prompt format
    #  example: {"role": "system", "content": "You are a helpful assistant."}
    # If your model supports json format, use the format parameter to specify that to your model.
    def ask(self, messages:list, format:str="", temperature:float=0.8):
        """
        Args:
            prompts (list): A list of prompts to ask.
            format (str, optional): The format of the response. Use "json" for json. Defaults to "".
            temperature (float, optional): The temperature of the LLM. Defaults to 0.8.
        """
        correct_format = is_correct_format(messages)
        if not correct_format:
            messages = convert_message_format(messages)
        # Preparation for inference
        text = processor.apply_chat_template(
            messages, tokenize=False, add_generation_prompt=True
        )
        image_inputs, video_inputs = process_vision_info(messages)
        inputs = processor(
            text=[text],
            images=image_inputs,
            videos=video_inputs,
            padding=True,
            return_tensors="pt",
        )
        inputs = inputs.to("cuda")
        
        # Inference: Generation of the output
        generated_ids = model.generate(**inputs, max_new_tokens=1024)
        generated_ids_trimmed = [
            out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
        ]
        output_text = processor.batch_decode(
            generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
        )
        if not correct_format:
            return output_text[0]
        return output_text

In [84]:
jarvis = JARVIS()
online_jarvis = OnlineAgent(jarvis)

In [81]:
c2 = [{'role': 'system', 'content': [{'type': 'text', 'text': 'Instructions:\nYou are a smart online searcher for a large language model.\nGiven information, you must create a search query to search the internet for relevant information.\n\nYour search query must be in the form of a json response.\n\nExact json response format must be as follows:\n\n{\n    "search_query": "your search query"\n}\n- Your must only provide ONE search query\n- You must provide the BEST search query for the given information\n- The search query must be normal text.\n'}]}, {'role': 'user', 'content': [{'type': 'text', 'text': 'What is the weather in Toronto today?'}]}]
print(jarvis.ask(c2))

[{'role': 'system', 'content': [{'type': 'text', 'text': 'Instructions:\nYou are a smart online searcher for a large language model.\nGiven information, you must create a search query to search the internet for relevant information.\n\nYour search query must be in the form of a json response.\n\nExact json response format must be as follows:\n\n{\n    "search_query": "your search query"\n}\n- Your must only provide ONE search query\n- You must provide the BEST search query for the given information\n- The search query must be normal text.\n'}]}, {'role': 'user', 'content': [{'type': 'text', 'text': 'What is the weather in Toronto today?'}]}]
After: 
[{'role': 'system', 'content': [{'type': 'text', 'text': 'Instructions:\nYou are a smart online searcher for a large language model.\nGiven information, you must create a search query to search the internet for relevant information.\n\nYour search query must be in the form of a json response.\n\nExact json response format must be as follows

In [86]:
content = "What is the weather in Toronto today? reply with today's date and temperature in celsius"
print(online_jarvis.search(content))

Based on information from the internet, the weather in Toronto today is 52°C with mostly cloudy conditions.


In [9]:
from pdf2image import convert_from_path
db = SQLiteFileSummaryStorage()
def generate_summary(filepath, query):
    jarvis = JARVIS()
    content = []
    if filepath.endswith(".jpg") or filepath.endswith(".jpeg"):
        content.append({
            "type":"image",
            "image":filepath,
            "resized_height": 280,
            "resized_width": 420,
        })
    else:
        images = convert_from_path(filepath)
        for i, page in enumerate(images):
            content.append({
                "type":"image",
                "image":page,
                "resized_height": 280,
                "resized_width": 420,
            })
    content.append({"type": "text", "text": f"""You are a document summarization assistant. Your task is to create a comprehensive, searchable summary of a file based on its contents and provided description. Follow this structured approach:
GUIDELINES:

Focus on discoverability - include terms users might search for
Incorporate both technical and layman's terms
Consider different ways users might try to find this document
Include crucial context from both the file content and user description
Add industry-standard terminology where relevant
Consider various use cases and access patterns
Maintain factual accuracy while optimizing for searchability

Now, analyze the provided file and description to generate a comprehensive summary like this, Do not exceed the numbers mentioned:

INPUTS:
File: attached pages as images. treat them combined as single file
User Description: {query}
Generate a summary in the following format:
CORE SUMMARY (2-3 sentences):
[Provide a clear, concise overview of the main content and purpose of the file]

DOCUMENT TYPE & PURPOSE:
- Primary Purpose: [Main function/goal of the document]
- Document Category: [Report/Specification/Manual/Analysis/etc.]
- Target Audience: [Intended readers/users]

KEY TOPICS:
- [Topic 1]
- [Topic 2]
- [Topic 3]
[List 3-5 main topics covered]

REFERENCE CONTEXTS:
"This file would be relevant when looking for..."
- [Scenario 1]
- [Scenario 2]
- [Scenario 3]
[List 3-4 specific use cases or scenarios]

ADDITIONAL CONTEXT FROM USER DESCRIPTION:
[Integrate any unique insights or context from the user's description that aren't apparent from the file content alone]

SEARCH KEYWORDS:
[List 8-12 relevant search terms, including:
- Common synonyms
- Technical terms
- Related concepts
- Alternative phrasings
Separate with commas]

"""})
    print(content)
    messages = [
        {
            "role": "user",
            "content": content,
        }
    ]
    
    output_text = jarvis.ask(messages)
    return output_text

def get_file(query):
    jarvis = JARVIS()
    summaries = db.get_all_summaries()
    print(len(summaries.items()))
    print(query)
    scores = ""
    for filename, data in summaries.items():
        summary = data['summary']
        path = data['filepath']
        print("Summary: " + summary)
        print("Query: " + query)
        messages = [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": f"""You are a document matching expert. Analyze how well a query matches a given summary and provide a single confidence score from 0-100.

Scoring Criteria:
1. Exact Keyword Matches (40 points max):
   - Exact matches of query terms (case-insensitive)
   - Multiple occurrences increase score
   - Technical/specific terms worth more than common words
   - Title/key position matches worth more

2. Keyword Variations (30 points max):
   - Plurals/singulars (car/cars)
   - Common synonyms (vehicle/automobile)
   - Word forms (run/running/ran)
   - Compound variations (healthcare/health care)

3. Semantic Relevance (20 points max):
   - Topic and concept alignment
   - Context relevance

4. Additional Factors (10 points max):
   - Term proximity
   - Information relevance

Query: {query}
Summary: {summary}

Follow these steps silently:
1. Check exact keyword matches
2. Find keyword variations
3. Assess semantic alignment
4. Consider additional factors
5. Sum up scores

Output only this format:
Confidence Score: <0-100>"""},
                ],
            }
        ]
        
        output_text = jarvis.ask(messages)
        scores += f"({filename},{path},{str(output_text[0])})\n"
    return scores

In [23]:
summary = generate_summary("Indian License.pdf", "My indian driving license")
db.save_summary("Indian License.pdf", "Indian License.pdf", summary[0])
print(get_file("my indian driving license"))

[{'type': 'image', 'image': <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1270x2000 at 0x7F9D40642550>, 'resized_height': 280, 'resized_width': 420}, {'type': 'text', 'text': 'You are a document summarization assistant. Your task is to create a comprehensive, searchable summary of a file based on its contents and provided description. Follow this structured approach:\nGUIDELINES:\n\nFocus on discoverability - include terms users might search for\nIncorporate both technical and layman\'s terms\nConsider different ways users might try to find this document\nInclude crucial context from both the file content and user description\nAdd industry-standard terminology where relevant\nConsider various use cases and access patterns\nMaintain factual accuracy while optimizing for searchability\n\nNow, analyze the provided file and description to generate a comprehensive summary like this, Do not exceed the numbers mentioned:\n\nINPUTS:\nFile: attached pages as images. treat them combined a

In [10]:
import re

def extract_function_pattern(response: str) -> tuple[bool, list[bool] | None]:
    """
    Check if the [boolean], [boolean], [boolean] pattern exists anywhere in the response
    and extract the boolean values if found.
    
    Args:
        response (str): The LLM's complete response string
    
    Returns:
        tuple[bool, list[bool] | None]: 
            - First element: True if pattern was found, False otherwise
            - Second element: List of 3 booleans if pattern was found, None otherwise
    
    Examples:
        >>> extract_function_pattern("To do this task, I need: [true], [false], [false]")
        (True, [True, False, False])
        >>> extract_function_pattern("Here's some regular text without pattern")
        (False, None)
    """
    # Pattern to match [boolean], [boolean], [boolean]
    pattern = r'\[(true|false)\],\s*\[(true|false)\],\s*\[(true|false)\]'
    
    # Search for pattern anywhere in the text
    match = re.search(pattern, response.lower())
    
    if match:
        # Extract the boolean values
        bool_values = [val == 'true' for val in match.groups()]
        return True, bool_values
    
    return False, None

In [18]:
import faiss
import numpy as np
import json
from sentence_transformers import SentenceTransformer
from typing import List, Dict
import os

class DocumentSearchSystem:
    def __init__(self, index_file: str = "faiss_index.bin", metadata_file: str = "metadata.json"):
        # Initialize the Sentence-BERT model for embedding generation
        self.model = SentenceTransformer('all-MiniLM-L6-v2')
        
        # File paths for persistence
        self.index_file = index_file
        self.metadata_file = metadata_file

        # List to store document metadata
        self.documents = []

        # Load or initialize FAISS index
        self.dimension = 384  # Embedding size of 'all-MiniLM-L6-v2'
        self.index = self._load_or_initialize_index()
        
        # Load document metadata
        self._load_metadata()
        
    def _load_or_initialize_index(self):
        """Load the FAISS index from disk or initialize a new one if not found."""
        if os.path.exists(self.index_file):
            index = faiss.read_index(self.index_file)
            print("FAISS index loaded from disk.")
        else:
            index = faiss.IndexFlatL2(self.dimension)  # L2 (Euclidean) index
            print("New FAISS index created.")
        return index

    def _load_metadata(self):
        """Load document metadata from JSON file if it exists."""
        if os.path.exists(self.metadata_file):
            with open(self.metadata_file, "r") as f:
                self.documents = json.load(f)
            print("Document metadata loaded from disk.")
        else:
            self.documents = []
    
    def _save_index_and_metadata(self):
        """Save the FAISS index and document metadata to disk."""
        # Save the FAISS index
        faiss.write_index(self.index, self.index_file)
        
        # Save the metadata as JSON
        with open(self.metadata_file, "w") as f:
            json.dump(self.documents, f)
        
        print("FAISS index and document metadata saved to disk.")

    def add_document(self, file_name: str, file_path: str, summary: str):
        """Add a document to the FAISS index and store its metadata."""
        # Generate the embedding for the summary
        embedding = self.model.encode(summary).astype(np.float32).reshape(1, -1)
        
        # Add embedding to FAISS index
        self.index.add(embedding)
        
        # Store metadata in documents list
        self.documents.append({
            "file_name": file_name,
            "file_path": file_path,
            "summary": summary
        })
        
        print(f"Added document: {file_name}")
        
        # Save index and metadata
        self._save_index_and_metadata()

    def search_documents(self, query: str, top_k: int = 3) -> List[Dict]:
        """Search for documents matching the query and return top_k results."""
        # Generate embedding for the query
        query_embedding = self.model.encode(query).astype(np.float32).reshape(1, -1)
        
        # Perform search in FAISS index
        distances, indices = self.index.search(query_embedding, top_k)
        
        # Retrieve and return matching documents
        results = []
        for idx, distance in zip(indices[0], distances[0]):
            if idx != -1:  # Ensure valid index
                doc = self.documents[idx]
                doc['score'] = 1 / (1 + distance)  # Convert distance to similarity score
                results.append(doc)
        
        return results

# # Usage example
# if __name__ == "__main__":
#     # Initialize the document search system
#     search_system = DocumentSearchSystem()
    
#     # Add some example documents
#     search_system.add_document("doc1.txt", "/path/to/doc1.txt", "This is a summary about machine learning.")
#     search_system.add_document("doc2.txt", "/path/to/doc2.txt", "An introduction to artificial intelligence.")
#     search_system.add_document("doc3.txt", "/path/to/doc3.txt", "A document detailing natural language processing.")
    
#     # Perform a search
#     query = "information about AI"
#     results = search_system.search_documents(query, top_k=2)
    
#     # Display results
#     for result in results:
#         print(f"File Name: {result['file_name']}")
#         print(f"File Path: {result['file_path']}")
#         print(f"Summary: {result['summary']}")
#         print(f"Score: {result['score']}")
#         print("-" * 40)


In [ ]:
app = Flask(__name__)

# Define upload folder
UPLOAD_FOLDER = '/content/uploads'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
search_system = DocumentSearchSystem()

@app.route('/upload', methods=['POST'])
def upload_file_and_text():
    # Get the text data from the request
    text = request.form['text']
#    jarvis = JARVIS()
#     online_jarvis = OnlineAgent(jarvis)
#     content = [
#         {
#             "role":"system",
#             "content":[
#                 {
#                     "type":"text",
#                     "text":"""You are Jarvis, Dishu's AI assistant. For each user query, your ONLY task is to determine if any of these functions are needed to properly respond:

# Available functions:
# 1. save_file: Saves data to a file
# 2. get_file: Reads data from a file  
# 3. use_internet: Accesses internet resources

# Response rules:
# 1. IF ANY function is needed:
#    - Respond ONLY with three boolean values, where AT LEAST ONE MUST BE TRUE: [save_file_needed], [get_file_needed], [use_internet_needed]
#    - Do not add any other text
#    Example responses:
#    - For saving content: [true], [false], [false]
#    - For reading saved content: [false], [true], [false] 
#    - For checking weather: [false], [false], [true]
#    - For tasks needing multiple functions: [true], [true], [false]

# 2. IF NO functions are needed:
#    - DO NOT use the boolean format
#    - Respond conversationally with the direct answer
#    - Example: "2+2 equals 4"

# Never output [false], [false], [false] - if no functions are needed, give a direct answer instead."""
#                 }
#             ]
#         },
#         {
#             "role":"user",
#             "content":[
#                 {
#                     "type":"text",
#                     "text":text
#                 }
#             ]
#         }
#     ]

#     output = jarvis.ask(content)

#     has_pattern, bool_values = extract_function_pattern(output[0])

#     if has_pattern:
#         save_file, retrieve_file, use_internet = bool_values
#         if save_file or retrieve_file:
            # Check if a file is part of the request
    if 'file' not in request.files:
        # scores = get_file(text)
        # print(scores)
        results = search_system.search_documents(text)
        output = {}
        i = 1
        for r in results:
            output[str(i)] = [r['file_name'], r['file_path'], r['score']]
            i += 1
        return jsonify({"text": output, "got_file": "true"}) 
    else:
        file = request.files['file']
        path = request.form['path']
        # Save the file securely
        file_path = os.path.join(app.config['UPLOAD_FOLDER'], file.filename)
        file.save(file_path)
    
        summary = generate_summary(file_path, text)
        print("Saving summary: " + summary)
        #db.save_summary(file.filename, path, summary)
        search_system.add_document(
                file_name=file.filename,
                file_path=path,
                summary=summary
            )
        print("Saved: " + path + "," + summary[0])
    
        # Return a response with the text and file path (for example purposes)
        output = {"text": "Saved: " + path, "file_saved": file_path}
        return jsonify(output)
        # if use_internet:
        #     output = online_jarvis.search(text)
        #     print("Internet: "+ output)
        #     return jsonify({"text": output})
    # print(output[0])
    # return jsonify({"text": output[0]})
            

@app.route('/download', methods=['GET'])
def download_file():
    # Get the filename from the query parameters
    filename = request.args.get('filename')
    
    # Construct the full path
    file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)

    # Check if the file exists
    if not os.path.isfile(file_path):
        abort(404)  # Return 404 if the file is not found

    # Send the file as an attachment
    return send_file(file_path, as_attachment=True)

app.run(host="0.0.0.0", port=4998)

FAISS index loaded from disk.
Document metadata loaded from disk.
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:4998
 * Running on http://192.168.84.2:4998
Press CTRL+C to quit
127.0.0.1 - - [22/Nov/2024 07:11:29] "POST /upload HTTP/1.1" 200 -
127.0.0.1 - - [22/Nov/2024 07:11:51] "POST /upload HTTP/1.1" 200 -
127.0.0.1 - - [22/Nov/2024 07:12:05] "POST /upload HTTP/1.1" 200 -
